# **Import Libraries**

In [ ]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.2/269.2 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib pandas flask twilio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving book-game-corner-d8d141ed5ac5.json to book-game-corner-d8d141ed5ac5.json


In [ ]:
import sqlite3
import datetime
import uuid
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from flask import Flask, jsonify
import telebot
import numpy as np
import random

# **Configuration**

In [ ]:
TELEGRAM_API_TOKEN = "7293862600:AAGdarl5Ov0qkxCKNci3f-6DN2ymtY78d3Q"
bot = telebot.TeleBot(TELEGRAM_API_TOKEN)

In [ ]:
app = Flask(__name__)

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SERVICE_ACCOUNT_FILE = '/content/book-game-corner-d8d141ed5ac5.json'
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=credentials)
SPREADSHEET_ID = '1FcD4uiMW2YBte6F63LwTQEup8rAavClTtlIGqr_vMgI'
RANGE_NAME = "'Form Responses 1'!B:F"

# **Database**

In [ ]:
conn = sqlite3.connect('game_corner.db', check_same_thread=False)
cursor = conn.cursor()

In [ ]:
def init_db():
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS students (
            name TEXT,
            nim TEXT PRIMARY KEY,
            program TEXT,
            email TEXT,
            telegram_id TEXT
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS bookings (
            booking_id TEXT PRIMARY KEY,
            student_nim TEXT,
            console_id TEXT,
            booking_time TEXT,
            queue_position INTEGER,
            FOREIGN KEY (student_nim) REFERENCES students (nim)
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS consoles (
            console_id TEXT PRIMARY KEY,
            name TEXT
        )
    """)
    conn.commit()

In [ ]:
def insert_dummy_students():
    students = [
       ("Nabila", "202410120123456", "Teknik Informatika", "nabila@student.ub.ac.id", "1109041774"),
        ("Fatiha", "202410120223457", "Teknik Informatika", "fatiha@student.ub.ac.id", "1487681153"),
        ("Laela", "202410120323458", "Teknik Informatika", "laela@student.ub.ac.id", "1168135991"),
        ("Karena", "202410120423459", "Teknik Informatika", "karena@student.ub.ac.id", "1109041774"),
        ("Amala", "202410120523460", "Teknik Informatika", "amala@student.ub.ac.id", "1487681153"),
        ("Fajri", "202410120623461", "Teknik Informatika", "fajri@student.ub.ac.id", "1168135991"),
        ("Salma", "202410120723462", "Teknik Informatika", "salma@student.ub.ac.id", "1109041774"),
        ("Ramadhianti", "202410120823463", "Teknik Informatika", "ramadhianti@student.ub.ac.id", "1487681153"),
        ("Zakir", "202410120923464", "Teknik Informatika", "zakir@student.ub.ac.id", "1168135991"),
        ("Lathani", "202410121023465", "Teknik Informatika", "lathani@student.ub.ac.id", "1109041774"),
        ("Pratama", "202410121123466", "Sistem Informasi", "pratama@student.ub.ac.id", "1487681153"),
        ("Aufa", "202410121223467", "Sistem Informasi", "aufa@student.ub.ac.id", "1168135991"),
        ("Dhila", "202410121323468", "Sistem Informasi", "dhila@student.ub.ac.id", "1487681153"),
        ("Sarah", "202410121423469", "Sistem Informasi", "sarah@student.ub.ac.id", "1487681153"),
        ("Rayan", "202410121523470", "Sistem Informasi", "rayan@student.ub.ac.id", "1168135991"),
        ("Hida", "202410121623471", "Sistem Informasi", "hida@student.ub.ac.id", "1487681153"),
        ("Roiyan", "202410121723472", "Sistem Informasi", "roiyan@student.ub.ac.id", "1168135991"),
        ("Bagas", "202410121823473", "Sistem Informasi", "bagas@student.ub.ac.id", "1487681153"),
        ("Akmal", "202410121923474", "Sistem Informasi", "akmal@student.ub.ac.id", "1487681153"),
        ("Ade", "202410122023475", "Sistem Informasi", "ade@student.ub.ac.id", "1168135991"),
        ("Arya", "202410122123476", "Teknik komputer", "arya@student.ub.ac.id", "1487681153"),
        ("Yusuf", "202410122223477", "Teknik komputer", "yusuf@student.ub.ac.id", "1487681153"),
        ("Azzam", "202410122323478", "Teknik komputer", "azzam@student.ub.ac.id", "1168135991"),
        ("Rama", "202410122423479", "Teknik komputer", "rama@student.ub.ac.id", "1487681153"),
        ("Sakti", "202410122523480", "Teknik komputer", "sakti@student.ub.ac.id", "1487681153"),
        ("Husain", "202410122623481", "Teknik komputer", "husain@student.ub.ac.id", "1168135991"),
        ("Ali", "202410122723482", "Teknik komputer", "ali@student.ub.ac.id", "14876811537"),
        ("Thalib", "202410122823483", "Teknik komputer", "thalib@student.ub.ac.id", "1109041774"),
        ("Agam", "202410122923484", "Teknik komputer", "agam@student.ub.ac.id", "1168135991"),
        ("Putra", "202410123023485", "Teknik komputer", "putra@student.ub.ac.id", "1487681153"),
        ("Aliando", "202410123123486", "Teknologi Informasi", "aliando@student.ub.ac.id", "1109041774"),
        ("Roufi", "202410123223487", "Teknologi Informasi", "roufi@student.ub.ac.id", "1168135991"),
        ("Sultan", "202410123323488", "Teknologi Informasi", "sultan@student.ub.ac.id", "1487681153"),
        ("Bambang", "202410123423489", "Teknologi Informasi", "bambang@student.ub.ac.id", "1168135991"),
        ("Daffa", "202410123523490", "Teknologi Informasi", "daffa@student.ub.ac.id", "1109041774"),
        ("Zaki", "202410123623491", "Teknologi Informasi", "zaki@student.ub.ac.id", "1168135991"),
        ("Nanda", "202410123723492", "Teknologi Informasi", "nanda@student.ub.ac.id", "1487681153"),
        ("Frandi", "202410123823493", "Teknologi Informasi", "frandi@student.ub.ac.id", "1109041774"),
        ("Andika", "202410123923494", "Teknologi Informasi", "andika@student.ub.ac.id", "1168135991"),
        ("Rahmat", "202410124023495", "Teknologi Informasi", "rahmat@student.ub.ac.id", "1487681153"),
    ]
    cursor.executemany("""
        INSERT OR IGNORE INTO students (name, nim, program, email, telegram_id)
        VALUES (?, ?, ?, ?, ?)
    """, students)
    conn.commit()

In [ ]:
def insert_dummy_consoles():
    consoles = [
        ("GC-001", "PlayStation 5"),
        ("GC-002", "Xbox"),
        ("GC-003", "Nintendo Switch"),
        ("GC-004", "PlayStation 5"),
        ("GC-005", "Xbox"),
    ]
    cursor.executemany("""
        INSERT OR IGNORE INTO consoles (console_id, name)
        VALUES (?, ?)
    """, consoles)
    conn.commit()

# **Reinforcement Learning**



In [ ]:
def get_all_consoles():
    result = cursor.execute("SELECT console_id FROM consoles").fetchall()
    print(f"Consoles available: {result}")  # Debugging
    return result


# Action: Menugaskan mahasiswa ke konsol tertentu
def take_action(console_id):
    cursor.execute("""
        SELECT c.console_id, COUNT(b.booking_id) as queue_count
        FROM consoles c
        LEFT JOIN bookings b ON c.console_id = b.console_id
        WHERE c.console_id = ?
        GROUP BY c.console_id
    """, (console_id,))
    return cursor.fetchone()


# Reward: Evaluasi alokasi
def get_reward(console_id, queue_position, start_time, end_time):
    max_closing_time = datetime.datetime.strptime("17:00", "%H:%M")
    if end_time > max_closing_time:
        return -10  # Penalti jika waktu bermain melampaui batas
    utilization = 10 if end_time <= max_closing_time else -5
    return utilization - queue_position  # Reward berdasarkan posisi antrean dan pemanfaatan

# State: [jumlah mahasiswa dalam antrean, waktu sekarang dalam menit, konsol yang tersedia]
def get_state():
    cursor.execute("SELECT COUNT(*) FROM bookings")
    queue_size = cursor.fetchone()[0]
    current_time = datetime.datetime.now()
    opening_time = datetime.datetime.strptime("08:00", "%H:%M")
    max_closing_time = datetime.datetime.strptime("17:00", "%H:%M")

    if current_time < opening_time:
        current_time = opening_time
    elif current_time > max_closing_time:
        current_time = max_closing_time

    elapsed_minutes = (current_time - opening_time).seconds // 60
    total_consoles = len(get_all_consoles())
    return np.array([queue_size, elapsed_minutes, total_consoles])

In [ ]:
# Hyperparameters
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.2  # Exploration rate

In [ ]:
class ReinforcementAgent:
    def __init__(self, num_states, num_actions, alpha=0.1, gamma=0.9, epsilon=0.2):
        self.q_table = np.zeros((num_states, num_actions))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    def choose_action(self, state):
        state_index = hash(tuple(state)) % len(self.q_table)
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(range(self.q_table.shape[1]))  # Exploration
        return np.argmax(self.q_table[state_index])  # Exploitation

    def update_q_table(self, state, action, reward, next_state):
      state_index = hash(tuple(state)) % self.q_table.shape[0]
      next_state_index = hash(tuple(next_state)) % self.q_table.shape[0]
      self.q_table[state_index, action] += self.alpha * (
          reward + self.gamma * np.max(self.q_table[next_state_index]) - self.q_table[state_index, action]
      )

# Inisialisasi agen
num_states = 100
num_actions = len(get_all_consoles())
agent = ReinforcementAgent(num_states, num_actions)


Consoles available: [('GC-001',), ('GC-002',), ('GC-003',), ('GC-004',), ('GC-005',)]


In [ ]:
def add_booking(student_nim):
    state = get_state()
    all_consoles = get_all_consoles()
    if not all_consoles:
        print("No consoles available for booking.")
        return None

    action = agent.choose_action(state)
    console_id = all_consoles[action][0]
    console = take_action(console_id)
    if not console:
        return None

    console_id, queue_count = console
    try:
        queue_count = int(queue_count)  # Mengubah nilai queue_count menjadi integer
    except ValueError:
        raise ValueError(f"Invalid queue_count: {queue_count}")

    booking_id = str(uuid.uuid4())
    opening_time = datetime.datetime.strptime("08:00", "%H:%M")
    max_closing_time = datetime.datetime.strptime("17:00", "%H:%M")

    global_position = queue_count
    play_start_time = opening_time + datetime.timedelta(hours=global_position // len(all_consoles))
    play_end_time = play_start_time + datetime.timedelta(hours=1)

    if play_start_time >= max_closing_time or play_end_time > max_closing_time:
        reward = get_reward(console_id, global_position, play_start_time, play_end_time)
        agent.update_q_table(state, action, reward, state)
        return None

    reward = get_reward(console_id, global_position, play_start_time, play_end_time)
    next_state = get_state()
    agent.update_q_table(state, action, reward, next_state)

    booking_time = play_start_time.strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute("""
        INSERT INTO bookings (booking_id, student_nim, console_id, booking_time, queue_position)
        VALUES (?, ?, ?, ?, ?)
    """, (booking_id, student_nim, console_id, booking_time, global_position + 1))
    conn.commit()

    return booking_id, console_id, global_position + 1, play_start_time.strftime("%H:%M"), play_end_time.strftime("%H:%M")


In [ ]:
def get_next_console():
    cursor.execute("""
        SELECT c.console_id, COUNT(b.booking_id) as queue_count
        FROM consoles c
        LEFT JOIN bookings b ON c.console_id = b.console_id
        GROUP BY c.console_id
        ORDER BY queue_count ASC, c.console_id ASC
        LIMIT 1
    """)
    result = cursor.fetchone()
    print(f"Query result: {result}")  # Debugging
    if result:
        console_id, queue_count = result
        return console_id, int(queue_count)
    return None


# **Telegram**

In [ ]:
def send_telegram_message(telegram_id, message):
    if not telegram_id.isdigit():
        print(f"Invalid telegram_id: {telegram_id}")
        return
    try:
        print(f"Sending to Telegram ID: {telegram_id} | Message: {message}")
        bot.send_message(telegram_id, message)
    except Exception as e:
        print(f"Error sending message to {telegram_id}: {e}")

# **Google Sheets**

In [ ]:
def process_google_sheets():
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    print("Data from Google Sheets:")
    for row in values:
        print(row)

    if not values:
        return jsonify({"message": "Tidak ada data di Google Sheets."}), 400

    results = []
    for row in values[1:]:
        if len(row) < 5:
            print(f"Skipping row with insufficient data: {row}")
            continue
        nama, nim, program, email, telegram_id = row[:5]
        cursor.execute("SELECT * FROM students WHERE nim = ?", (nim,))
        student = cursor.fetchone()
        if student:
            booking_data = add_booking(nim)
            if booking_data:
                booking_id, console_id, queue_position, start_time, end_time = booking_data
                message = f"Hi {nama}, booking berhasil!\nDetail:\n- ID: {booking_id}\n- Konsol: {console_id}\n- Posisi: {queue_position}\n- Waktu: {start_time} - {end_time}"
                send_telegram_message(telegram_id, message)
                results.append({"nim": nim, "status": "Berhasil", "booking_id": booking_id})
            else:
                send_telegram_message(telegram_id, f"Hi {nama}, booking gagal. Tidak ada konsol tersedia.")
                results.append({"nim": nim, "status": "Gagal"})
        else:
            send_telegram_message(telegram_id, f"Hi {nama}, Anda tidak terdaftar.")
            results.append({"nim": nim, "status": "Tidak Terdaftar"})
    return jsonify(results)

# **Main**

In [ ]:
if __name__ == '__main__':
    init_db()
    insert_dummy_students()
    insert_dummy_consoles()

    num_consoles = len(get_all_consoles())
    num_states = 100
    agent = ReinforcementAgent(num_states, num_consoles)

    with app.app_context():
        process_google_sheets()
    app.run()


Consoles available: [('GC-001',), ('GC-002',), ('GC-003',), ('GC-004',), ('GC-005',)]
Data from Google Sheets:
['nama', 'nim', 'program', 'email', 'telegram_id']
['Fatiha', '202410120223457', 'Teknik Informatika', 'fatiha@student.ub.ac.id', '1487681153']
['Laela', '202410120323458', 'Teknik Informatika', 'laela@student.ub.ac.id', '1109041774']
['Nabila', '202410120123456', 'Teknik Informatika', 'nabila@student.ub.ac.id', '1109041774']
['Amala', '202410120523460', 'Teknik Informatika', 'amala@student.ub.ac.id', '1109041774']
['Nabila', '202410120123456', 'Teknik Informatika', 'nabila@student.ub.ac.id', '1109041774']
['Nabila', '202410120123456', 'Teknik Informatika', 'nabila@student.ub.ac.id', '1109041774']
['Laela ', '202410120323458', 'Teknik Informatika', 'laela@student.ub.ac.id', '1168135991']
['Nabila', '202410120123456', 'Teknik Informatika', 'nabila@student.ub.ac.id', '1109041774']
['Joko', '202410123723111', 'Teknik Informatika', 'joko@student.ub.ac.id', '1487681153']
['Rahmat',

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
